In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
assistant_id = "asst_gLmlNp9EnQaeyt9ale3kpOlc"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_AjxSoC0AS93LXEMqPLnc1Dy8', created_at=1735383320, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_KEdK2lHPo0QbiAR9JrbreTvA', assistant_id='asst_gLmlNp9EnQaeyt9ale3kpOlc', cancelled_at=None, completed_at=None, created_at=1735383410, expires_at=1735384010, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_AjxSoC0AS93LXEMqPLnc1Dy8', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [6]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [11]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_KEdK2lHPo0QbiAR9JrbreTvA', assistant_id='asst_gLmlNp9EnQaeyt9ale3kpOlc', cancelled_at=None, completed_at=None, created_at=1735383410, expires_at=1735384010, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1735383436, status='queued', thread_id='thread_AjxSoC0AS93LXEMqPLnc1Dy8', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=F

In [14]:
get_run(run.id, thread.id).status

'completed'

In [15]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: To analyze whether Salesforce (CRM) stock is a good buy, we can consider several factors including the company's financial health, stock performance, and market conditions. Here's the information we have:

### Financial Health
From the most recent income statement, we have the following key figures:
- Total Revenue: $34.857 billion
- Cost of Revenue: $8.541 billion
- Gross Profit: $26.316 billion
- Net Income: $4.136 billion
- Operating Income: $5.999 billion
- Research and Development Expenses: $4.906 billion

The balance sheet shows:
- Total Assets: $99.823 billion
- Total Liabilities: $40.177 billion
- Total Equity: $59.646 billion
- Long Term Debt: $8.427 billion

### Stock Performance
Over the last 100 days, the stock has seen fluctuations with some peaks and troughs. Notably, there's a dividend payout of $0.4 which indicates the company is returning value to shareholders. The stock has had significant movement 